In [ ]:
%load_ext kamu

In [ ]:
%import_dataset acme.fishing.co/vessels.gps --alias gps
%import_dataset acme.fishing.co/vessels.trawl --alias trawl
%import_dataset acme.fishing.co/vessels.fuel --alias fuel
%import_dataset acme.fishing.co/vessels.location-annotated --alias loc
%import_dataset globalfishingwatch.org/protected-areas --alias areas

In [ ]:
%%local
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from mapboxgl.viz import *
from mapboxgl.utils import *
from utils.plotting import *

# Must be a public token, starting with `pk`
token = os.getenv('MAPBOX_ACCESS_TOKEN')

mapbox_style = 'mapbox://styles/mapbox/streets-v11'

# Vessel Activity

In [ ]:
%%sql -o gps -q
select
    event_time,
    vessel_name,
    longitude,
    latitude,
    is_trawling
from loc

In [ ]:
%%local
fig = go.Figure()

for vessel_name in gps['vessel_name'].unique():
    df = gps[gps.vessel_name == vessel_name]
    fig.add_trace(
        go.Scattermapbox(
            name=vessel_name,
            mode="markers+lines",
            lat=df["latitude"],
            lon=df["longitude"],
        ),
    )

fig.update_layout(
    height=700,
    margin={"r":0,"t":0,"l":0,"b":0},
    hovermode='closest',
    mapbox=dict(
        style=mapbox_style,
        accesstoken=token,
        bearing=0,
        center=go.layout.mapbox.Center(lat=51, lon=2),
        pitch=0,
        zoom=7,
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()


# Protected Marine Areas

In [ ]:
%%sql -o areas -q
select
    geometry,
    date,
    wdpa_pid,
    gis_area
from areas
-- NOTE: this is a temporary hack due to upstream Spark issue: https://issues.apache.org/jira/browse/SPARK-47150
order by len(geometry)
limit 5500

In [ ]:
%%local
viz = ChoroplethViz(
    df_to_geojson(areas),
    style=mapbox_style,
    center=(2, 51),
    zoom=5,
    access_token=token,
    color_property='gis_area',
    color_stops=create_color_stops([0], colors=['#0000aa']),
    color_default='white',
    line_width=0,
    opacity=0.4,
)

viz.show()

# Spatial Queries

In [ ]:
%%sql -o isect_areas -q
select
    wdpa_pid, 
    parent_iso, 
    desig_eng, 
    name,
    gis_area,
    geometry
from areas where parent_iso in ("NLD", "FRA", "DMK", "BEL")

In [ ]:
%%sql -o isect -q
select
    gps.event_time,
    gps.vessel_name,
    gps.latitude,
    gps.longitude
from (
    select
         event_time, vessel_name, latitude, longitude, st_point(longitude, latitude) as geometry 
    from loc where is_trawling = 1
) gps,
(
    select
        st_geomfromgeojson(geometry) as geometry
    from areas where parent_iso = "NLD"
) areas
where st_contains(areas.geometry, gps.geometry)

In [ ]:
%%local
isect_areas_geojson = df_to_geojson(isect_areas)

fig = go.Figure()

fig.add_trace(go.Choroplethmapbox(
    geojson=isect_areas_geojson,
    locations=isect_areas.wdpa_pid,
    featureidkey="properties.wdpa_pid",
    z=isect_areas.gis_area,
    colorscale="Viridis",
    marker_line_width=0,
    marker_opacity=0.1,
    text=isect_areas.name,
    name="Protected Area",
    showscale=False,
))

for vessel_name in isect['vessel_name'].unique():
    df = isect[isect.vessel_name == vessel_name]
    fig.add_trace(
        go.Scattermapbox(
            name=vessel_name,
            mode="markers+lines",
            lat=df["latitude"],
            lon=df["longitude"],
        ),
    )

fig.update_traces(showlegend=False, selector=dict(type='choroplethmapbox'))

fig.update_layout(
    height=800,
    margin={"r":0,"t":0,"l":0,"b":0},
    mapbox=dict(
        style=mapbox_style,
        accesstoken=token,
        zoom=8,
        center={"lat": 51.5, "lon": 3.5}
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)
fig.show()